In [1]:
import tensorflow as tf
import numpy as np

/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/i550012/virtualenv/tensorflow1

In [14]:
bandits = [0.3, 0.4, 0.7, 0.5]
num_bandits = len(bandits)

In [3]:
def pullBandit(bandit):
    rand = np.random.random()
    return 1 if rand < bandit else -1    

In [4]:
tf.reset_default_graph()
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights)
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights, action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(0.001)
update = optimizer.minimize(loss)

In [15]:
total_episodes = 2000
total_reward = np.zeros(num_bandits)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
e = 0.1
i = 0
while i < total_episodes:
    if np.random.rand(1) > e:
        action = sess.run(chosen_action)
    else:
        action = np.random.randint(num_bandits)
    
    reward = pullBandit(bandits[action])
    
    _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict={reward_holder:[reward], action_holder:[action]})
    total_reward[action] += reward
    i+=1
    if i%100 == 0:
        print("Running reward after " + str(i) + " episodes: " + str(total_reward))

print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
print("The known best bandit is bandit " + str(np.argmax(bandits)+1))

Running reward after 100 episodes: [-3. -4. 41. -2.]
Running reward after 200 episodes: [-2. -3. 76. -3.]
Running reward after 300 episodes: [ -4.  -2. 109.  -3.]
Running reward after 400 episodes: [ -4.  -6. 136.  -2.]
Running reward after 500 episodes: [ -4.  -8. 180.  -2.]
Running reward after 600 episodes: [ -6.  -8. 218.  -2.]
Running reward after 700 episodes: [ -7.  -8. 266.  -1.]
Running reward after 800 episodes: [ -8. -10. 315.  -1.]
Running reward after 900 episodes: [ -9. -13. 346.   0.]
Running reward after 1000 episodes: [ -9. -12. 381.  -2.]
Running reward after 1100 episodes: [-11. -12. 405.   2.]
Running reward after 1200 episodes: [-15. -13. 443.   5.]
Running reward after 1300 episodes: [-17. -16. 476.   5.]
Running reward after 1400 episodes: [-18. -19. 510.   7.]
Running reward after 1500 episodes: [-19. -23. 542.   6.]
Running reward after 1600 episodes: [-18. -24. 591.   7.]
Running reward after 1700 episodes: [-19. -25. 627.   7.]
Running reward after 1800 episo